In [1]:
#download price data
import yfinance as yf

#base libs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#scaler and split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

#models
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

#to show accuracy
from sklearn.metrics import classification_report

#шоб варнінги убрати ато супер бісить
import warnings
warnings.filterwarnings('ignore')

In [29]:
data = yf.download('JPYUSD=X', start='2024-01-01', end='2025-11-04') #завантажую данні

[*********************100%***********************]  1 of 1 completed


In [4]:
data.head()

Price,Close,High,Low,Open,Volume
Ticker,JPYUSD=X,JPYUSD=X,JPYUSD=X,JPYUSD=X,JPYUSD=X
Date,,,,,
2024-01-01,0.007095,0.007095,0.007091,0.007095,0
2024-01-02,0.007086,0.007087,0.007033,0.007086,0
2024-01-03,0.007035,0.007049,0.006959,0.007035,0
2024-01-04,0.006993,0.006999,0.006905,0.006993,0
2024-01-05,0.006908,0.006952,0.006851,0.006908,0


In [30]:
df = data[['Open', 'High', 'Low', 'Close', 'Volume']].copy() #копіюю данні з дати в датафрейм

In [31]:
df['Volume'].value_counts(normalize=True) #анлак

JPYUSD=X
0           1.0
Name: proportion, dtype: float64

підготовка данних

In [33]:
period = 14 #це нада для RSI та Stochastic Oscillator

In [34]:
df['MA_5'] = df['Close'].rolling(window=5).mean()  # Середня за 5 днів
df['MA_20'] = df['Close'].rolling(window=20).mean()  # Середня за 20 днів
df['MA_50'] = df['Close'].rolling(window=50).mean() # Середня за 50 днів
df['MA_200'] = df['Close'].rolling(window=200).mean() #середня за 200 днів
# Якщо ціна вище MA_20  → купувати (восходящий тренд)
# Якщо ціна нижче MA_20 → продавати (нисходящий тренд)
# Якщо MA_5 перетинає MA_20 вгору → СИГНАЛ ПОКУПКИ
# Якщо MA_5 перетинає MA_20 вниз → СИГНАЛ ПРОДАЖУ

In [35]:
df['Price_Change'] = df['Close'].pct_change()  # Відсоток змін, pct_change() pandas обчислює відсоткову зміну між поточним і попереднім значенням в серії або колонці
df['ATR'] = df['Price_Change'].rolling(window=10).std()  # коливання ціни, волативність std() стандартне відхилення

In [36]:
rolling_std = df['Close'].rolling(window=20).std() #стандартне відхилення
rolling_std = rolling_std.squeeze() #тут змінюю розмірність на 1Д
df['Middle Band'] = df['MA_20']
df['Upper Band'] = df['MA_20'] + (2 * rolling_std)
df['Lower Band'] = df['MA_20'] - (2 * rolling_std)
# Якщо ціна торгується біля нижньої смуги → ПЕРЕПРОДАНО (можна купувати)
# Якщо ціна торгується біля верхної смуги → ПЕРЕКУПЛЕНО (можна продавати)
# Вузькі смуги → низька волатильність (спокій)
# Широкі смуги → висока волатильність (рух)

In [11]:
#RSI
delta = df['Close'].diff() #знаходить різницю клоус
gain = (delta.where(delta > 0, 0)).rolling(window=period).mean() #перевіряє де дельта більше нуля, рахує середній прибуток
loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean() #де дельта менше, рахує середній збиток
rs = gain / loss #підраховує відносн силу тренда
rsi = 100 - (100 / (1 + rs)) #RSI (0-100) — показує перекупленість/перепроданість
df['RSI'] = rsi

In [12]:
#Stochastic Oscillator
lowest_low = df['Low'].rolling(window=period).min() #розраховує найнижчий мінімум
highest_high = df['High'].rolling(window=period).max() #розраховує найвищий максимум
df['StochasticOscillatorK'] = 100 * (df['Close'] - lowest_low) / (highest_high - lowest_low)
#Лінія %K стохастичного осцилятора — це швидка лінія індикатора, яка показує,
#наскільки поточна ціна закриття активу близька до верхньої або нижньої межі цінового діапазону за певний період
df['StochasticOscillatorD'] = df['StochasticOscillatorK'].rolling(window=3).mean() 
#лінії стохастичного осцилятора %K, яка згладжує її коливання

In [13]:
df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
#функція shift() з параметром -1, зсуває усі значення колонки Close вгору на 1 рядок, для кожного рядка тепер стоїть ціна закриття наступного дня.
#Якщо ціна завтра більша за сьогодні — умова True

In [37]:
df = df.dropna() #видалив пропуски

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 279 entries, 2024-10-04 to 2025-11-03
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   (Open, JPYUSD=X)           279 non-null    float64
 1   (High, JPYUSD=X)           279 non-null    float64
 2   (Low, JPYUSD=X)            279 non-null    float64
 3   (Close, JPYUSD=X)          279 non-null    float64
 4   (Volume, JPYUSD=X)         279 non-null    int64  
 5   (MA_5, )                   279 non-null    float64
 6   (MA_20, )                  279 non-null    float64
 7   (MA_50, )                  279 non-null    float64
 8   (MA_200, )                 279 non-null    float64
 9   (Price_Change, )           279 non-null    float64
 10  (ATR, )                    279 non-null    float64
 11  (Middle Band, )            279 non-null    float64
 12  (Upper Band, )             279 non-null    float64
 13  (Lower Band, )             279 

In [16]:
features = ['Close', 'High', 'Low', 'Volume', 'MA_5', 'MA_20', 'MA_50', 'MA_200', 'ATR', 'RSI', 'Price_Change', 'Middle Band',
            'Upper Band', 'Lower Band', 'ATR', 'StochasticOscillatorK', 'StochasticOscillatorD']
X = df[features]
y = df['Target']
#ну тут просто розділив на фічі і назначив Х і Y

In [17]:
scaler_features = MinMaxScaler()
X_scaled = scaler_features.fit_transform(X)
#нормалізував значення, +поняв шо круче не перезаписувати данні а робити нью змінні(вроде круче)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.15, random_state=42)
#розділив данні на 80% на тренування, 20 на тест

In [19]:
len(X_train)

237

In [20]:
len(X_test)

42

In [38]:
model = LogisticRegression(max_iter=10000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'lbfgs'
,max_iter,10000
,multi_class,'deprecated'


In [39]:
# Передбачення на тренувальному наборі
y_train_pred = model.predict(X_train)

# Передбачення на тестовому наборі
y_test_pred = model.predict(X_test)

In [40]:
# Передбачення на тренувальному наборі
print(classification_report(y_train, y_train_pred, 
    target_names=['Ціна вниз', 'Ціна вгору']))

              precision    recall  f1-score   support

   Ціна вниз       0.69      0.57      0.62       114
  Ціна вгору       0.66      0.76      0.71       123

    accuracy                           0.67       237
   macro avg       0.67      0.67      0.67       237
weighted avg       0.67      0.67      0.67       237



In [41]:
# Передбачення на тестовому наборі
print(classification_report(y_test, y_test_pred, 
    target_names=['Ціна вниз', 'Ціна вгору']))
#нова фіча, target_names, просто зручніше бачити що модель норм предіктить а шо не

              precision    recall  f1-score   support

   Ціна вниз       0.75      0.55      0.63        22
  Ціна вгору       0.62      0.80      0.70        20

    accuracy                           0.67        42
   macro avg       0.68      0.67      0.66        42
weighted avg       0.69      0.67      0.66        42



In [25]:
model = XGBClassifier()
model.fit(X_train, y_train)

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [26]:
# Передбачення на тренувальному наборі
y_train_pred_XGB = model.predict(X_train)

# Передбачення на тестовому наборі
y_test_pred_XGB = model.predict(X_test)

In [27]:
# Передбачення на тренувальному наборі
print(classification_report(y_train, y_train_pred_XGB, 
    target_names=['Ціна вниз', 'Ціна вгору']))

              precision    recall  f1-score   support

   Ціна вниз       1.00      1.00      1.00       114
  Ціна вгору       1.00      1.00      1.00       123

    accuracy                           1.00       237
   macro avg       1.00      1.00      1.00       237
weighted avg       1.00      1.00      1.00       237



In [28]:
# Передбачення на тестовому наборі
print(classification_report(y_test, y_test_pred_XGB, 
    target_names=['Ціна вниз', 'Ціна вгору']))

              precision    recall  f1-score   support

   Ціна вниз       0.71      0.55      0.62        22
  Ціна вгору       0.60      0.75      0.67        20

    accuracy                           0.64        42
   macro avg       0.65      0.65      0.64        42
weighted avg       0.66      0.64      0.64        42

